# A570 Final Project (Project #1)
### Armaan Goyal
### Updated 3/18/2020

This notebook contains all relevant code for Project #1, which mainly consists of assesing the structure and dynamics of star clusters by generating 3D and projected 2D King (King 1966) and Wilson (Wilson 1975) density and velocity dispersion profiles and fitting them to the surface brightness profiles of 68 nearby clusters (McLaughlin & van der Marel 2005). 

The code will be structured the same as the associated written report, whereby the first half of the analysis consists of generation of the theoretical models, and the second half is concerned with fitting the models to the observed data.

## Generation of Theoretical Models

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
from scipy.integrate import quad
from scipy.optimize import curve_fit
import pandas as pd
from scipy.integrate import odeint
from scipy.interpolate import interp1d

# plotting and LaTeX
from astropy.visualization import astropy_mpl_style
plt.style.use("ggplot")
mpl.rcParams["figure.dpi"]=100
%config InlineBackend.figure_format = "svg"
plt.rcParams["text.usetex"] = True
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

Define equations for computation of 3D density and velocity dispersion profiles:

In [ ]:
#define main equations
def gamma_inc(s, x):
    '''
   Incomplete gamma function.
    
    '''
    def integrand(t):
        if t >= 0:
            return (t**(s-1))*np.exp(-t)
        else:
            return 0
    return quad(integrand, 0, x)[0]

gamma_vec = np.vectorize(gamma_inc, excluded = ["s"]) # array-compatible gamma function

def rho_3D(W, W0, prof = "K"): # 3D density profiles rho(W)
    '''
    Generate 3D density profile.
    
    Parameters:
    -----------
    W (float): Dimensionless potential
    W0 (float): Dimensionless central potential
    prof (str): "K" for King profile (default), "W" for Wilson profile
    
    Returns:
    --------
    Computes 3D density profile (array) for specified profile type.
    
    '''
    if prof == "K":
        return (np.exp(W)*gamma_vec(5/2, W))/(np.exp(W0)*gamma_vec(5/2, W0))
    if prof == "W":
        return (np.exp(W)*gamma_vec(7/2, W))/(np.exp(W0)*gamma_vec(7/2, W0))

def WR_ODE(U, R, W0, prof):
    '''
    Define ODE corresponding to Poisson's equation for W(r).
    
    Parameters:
    -----------
    U (list): State vector [W(r), W'(r)]
    R (float): Radius
    See previous functions for W0 and prof
    
    Returns:
    --------
    Derivative vector [W'(r), W''(r)] (list)
    
    '''
    if prof == "K":
        return [U[1], -1*(2/R)*U[1] - 9*rho_3D(U[0], W0, prof)]
    if prof == "W":
        return [U[1], -1*(2/R)*U[1] - 9*rho_3D(U[0], W0, prof)]

def velocity_3D(W0, W, prof):
    '''
    Calculate 3D velocity dispersion.
    
    Parameters:
    -----------
    See previous functions.
    
    Returns:
    --------
    3D velocity profile (array).
    
    '''
    
    if prof == "K":
        s1, s2 = 7/2, 5/2 # define gamma indices for profile type
    if prof == "W":
        s1, s2 = 9/2, 7/2 
    vels = (gamma_vec(s1, W)/gamma_vec(s2, W))/(gamma_vec(s1, W0)/gamma_vec(s2, W0))
    return vels

def make_3D_profile(W0, prof, make_vel = False):
    '''
    Generate 3D density profile rho(r) and with rho_3D and WR_ODE. 
    Option for also generating 3D velocity dispersion profile with velocity_3D.
    
    Parameters:
    -----------
    See previous functions for W0 and prof
    make_vel = Boolean toggle for outputting 3D velocity profile (default is False)
    
    Returns:
    --------
    Radii (array), their corresponding 3D density values (array), and tidal radius (float). 
    Also outputs 3D velocity dispersion profile (array) if desired.
    
    '''
    U0 = np.array([W0, 0]) # define initial state vector
    Rs = np.logspace(-2, 2.2, 1000) # generate lots of radial coordinates 
    Us = odeint(WR_ODE, U0, Rs, args=(W0,prof)) # solve Poisson's equation for W(r)
    Ws = Us[:, 0]
    pos_mask = (Ws >= 0) # only take values with W > 0
    Ws = Ws[pos_mask]
    Rs = Rs[pos_mask]
    Rt = Rs[-1] # pick outermost radial value for tidal radius
    rhos = rho_3D(Ws, W0, prof) # make density profile
    rhos_normed = rhos/rhos[0] # normalize
    if make_vel:
        vels = velocity_3D(W0, Ws, prof) # make velocity profiles
        return Rs, rhos_normed, Rt, vels
    else:
        return Rs, rhos_normed, Rt

Define interpolation function and projection integral to calculate 2D profiles.

In [ ]:
def make_interp_profile(x, y):
    '''
    Generate interpolating polynomial for some arrays x and y.
    
    '''
    poly = interp1d(x, y, fill_value = "extrapolate")
    return poly

def make_2D_profile(R, Rt, poly):
    '''
    Compute projected 2D profile for given 3D profile.
    (WARNING: This function uses scipy.integrate.quad to integrate over a somewhat dubious 
    range and will therefore output some error messages concerning extrapolation and roundoff 
    error to warn the user about this. We maintain that the result is numerically sound and
    accurate for the purposes of this analysis.)
    
    Parameters:
    -----------
    R (float): Radius
    Rt (float): Tidal radius, upper bound for integral
    poly (poly object): Interpolating polynomial for 3D profile.
    
    Returns:
    --------
    2D projected profile at input radius (float)
    
    '''
    func = lambda r: (poly(r)*r)/np.sqrt(r*r - R*R) # define integrand
    integral = 2*quad(func, R, Rt)[0] # integrate to tidal radius.
    return integral

# vectorize make_2D_profile so that it can take in/output arrays
make_surf_vec = np.vectorize(make_2D_profile, excluded = ["Rt", "poly"])

The cell below performs the computation of all relevant 3D and 2D profiles (as well as half-light and tidal radii) for each value of $W_{0}$ in a user-defined list.

We note that while it is generally considered better coding practice to separate the generation and plotting of these profiles, we will perform these tasks simultaneously here to avoid the even lengthier computation times imposed by having to save and maintain each model. In this implementation, the current plotting routine will be agnostic to all but the current model in the loop, thereby avoiding unnecessary memory usage.

In [ ]:
W0_list = [2, 4, 6, 8] # define values of W for profiles
prof_list = ["K", "W"]
color_list = ["tab:red", "tab:blue", "tab:green", "tab:orange"] # colors for each W

Rh_w_list = [] # define lists for half-light and tidal radii
Rh_k_list = []
Rt_w_list = []
Rt_k_list = []

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (8, 10), sharex = True) # build subplots

for i in range(len(W0_list)): # iterate over Ws
    for j in range(len(prof_list)): # for each W consider both King and Wilson models
        Rs, rhos, Rt, vels = make_3D_profile(W0_list[i], prof_list[j], make_vel = True) # make 3D profiles
        poly3D = make_interp_profile(Rs, rhos) # make interpolating polynomials
        polyvels = make_interp_profile(Rs, vels)
        sigs = make_surf_vec(Rs, Rt, poly3D) # generate 2D densities
        sigs = np.nan_to_num(sigs, copy = False) # eliminate NaNs if any happen to be present
        vel2D = make_surf_vec(Rs, Rt, polyvels) # generate 2D velocity dispersion profile
        vel2D = np.nan_to_num(vel2D, copy = False) 
        vel2D = vel2D/vel2D[0] # normalize
        inv_poly2D = make_interp_profile(sigs, Rs) # make inverse interpolant for 2D density
        Rh = inv_poly2D(.5*sigs[0]) # interpolate for half-light radius
    
        if prof_list[j] == "K": # Plot King profiles as solid lines
            Rh_k_list.append(Rh)
            Rt_k_list.append(Rt)
            ax1.plot(Rs, rhos, label = "King, $W_{0}$ = %d"%W0_list[i], c = color_list[i])
            ax2.plot(Rs, sigs, c = color_list[i])
            ax3.plot(Rs, vels, c = color_list[i])
            ax4.plot(Rs, vel2D, c = color_list[i])
        
        if prof_list[j] == "W": # Plot Wilson profiles as dashed lines
            Rh_w_list.append(Rh)
            Rt_w_list.append(Rt)
            ax1.plot(Rs, rhos, ls = "--", label = "Wilson, $W_{0}$ = %d"%W0_list[i], c = color_list[i])
            ax2.plot(Rs, sigs, ls = "--", c = color_list[i])
            ax3.plot(Rs, vels, ls = "--", c = color_list[i])
            ax4.plot(Rs, vel2D, ls = "--", c = color_list[i])
            
legend_elements = [Line2D([0], [0], color='tab:red', lw=4, label='$W_{0}=2$'),
                   Line2D([0], [0], color='tab:blue', lw=4, label='$W_{0}=4$'),
                   Line2D([0], [0], color='tab:green', lw=4, label='$W_{0}=6$'),
                   Line2D([0], [0], color='tab:orange', lw=4, label='$W_{0}=8$'),
                   Line2D([0], [0], color='black', lw=3, label='King'),
                   Line2D([0], [0], color='black', ls = "--", lw=3, label='Wilson')]
    
# log-log plotting with axis labels and arbitrary bounds for clean plots
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_ylabel(r"log($\rho/\rho_{0})$", fontsize = 14)
ax1.set_ylim(1e-6, 3)
ax1.legend(handles=legend_elements, fontsize="large")

ax2.set_yscale('log')
ax2.set_xscale('log')
ax2.set_ylabel(r"log($\Sigma/r_{0}\rho_{0}$)", fontsize = 14)
ax2.set_ylim(1e-6, 3)

ax3.set_yscale('log')
ax3.set_xscale('log')
ax3.set_ylabel(r"log($\overline{v}^{2}/\overline{v}_{0}^{2})$", fontsize = 14)
ax3.set_ylim(1e-3, 3)

ax4.set_yscale('log')
ax4.set_xscale('log')
ax4.set_ylabel(r"log($\overline{\sigma_{p}}^{2}/\overline{\sigma_{p, 0}}^{2})$", fontsize = 14)
ax4.set_ylim(1e-3, 3)

ax4.tick_params(axis='x', labelsize= 13)
ax1.tick_params(axis='y', labelsize= 13)
ax2.tick_params(axis='y', labelsize= 13)
ax3.tick_params(axis='y', labelsize= 13)
ax4.tick_params(axis='y', labelsize= 13)

plt.xlabel(r"log$(r/r_{0})$", fontsize = 14)
plt.tight_layout()
fig.subplots_adjust(wspace=.03, hspace=.08)
plt.show()
fig.savefig("King_Wilson", dpi=400) # save figure

Make data frame for structural parameters (half-light radius and tidal radius) corresponding to each profile type and value of $W_{0}$.

In [ ]:
df_list = []

for i in range(len(W0_list)):
    k_list = [W0_list[i], "King", float(Rh_k_list[i]), Rt_k_list[i]] # King parameters
    w_list = [W0_list[i], "Wilson", float(Rh_w_list[i]), Rt_w_list[i]] # Wilson parameters
    
    df = pd.DataFrame([k_list, w_list], columns=['W0', 'Profile', 'Rh', 'Rt'])
    df = df.set_index(df.columns[0])
    df_list.append(df) # make data frame for each W0 

model_df = pd.concat(df_list) # combine data frames for all values of W0
model_df = model_df.round(decimals = 2)
model_df.to_latex() # output as LaTeX table

## Fitting Models to Observed Cluster Data

In [ ]:
def make_templates(W0_min, W0_max, res, prof):
    '''
    Compute templates for dimensionless 2D density models for given W0 bounds, resolution, 
    and profile type.
    
    Note: Filenames will have W0 expressed without a decimal 
    (e.g. King model for W0 = 1.4 will be saved as king_14.txt)
    
    Parameters:
    -----------
    W0_min, W0_max (float): Inclusive bounds for W0
    res (float): Resolution for W0
    prof: Profile type
    
    Returns:
    --------
    Prints 
    Saves .txt file containing template data
    
    '''
    W0_list = np.arange(W0_min, W0_max+res, res)
    
    for W0 in W0_list:
        Rs, rhos, Rt = make_3D_profile(W0, prof) # make 3D model
        poly3D = make_interp_profile(Rs, rhos)
        sigs = make_surf_vec(Rs, Rt, poly3D)
        sigs = np.nan_to_num(sigs, copy = False) # make 2D model
        
        # naming and saving .txt files
        output = np.column_stack((Rs, sigs))
        if prof == "K":
            name = "king_{}".format(str(int(W0*10)))
            np.savetxt(name, output)
            print("Generated dimensionless King model for W0 = {}. File saved as {}".format(W0, name))
            
        if prof == "W":
            name = "wilson_{}".format(str(int(W0*10)))
            np.savetxt(name, output)
            print("Generated dimensionless Wilson model for W0 = {}. File saved as {}".format(W0, name))

In [ ]:
# generate King templates
make_templates(1, 10, 0.1, "K")

In [ ]:
# generate Wilson templates
make_templates(1, 10, 0.1, "W")

Import cluster data from McLaughlin & van der Marel 2005 (.csv file provided in GitHub repository). We will be following their convention for the exclusion of certain flagged points from our fits.

In [ ]:
df = pd.read_csv("cluster_data_raw.csv") # import data
df.columns = ["cluster", "r", "log(r)", "mu", "mu_max", "mu_min", "band", "AV", "e_AV", "flag"]
df = df.set_index("cluster")
df = df[df["flag"] == 1] # exclude flagged points
clusters = list(df.index.unique()) # cluster names 

for cluster in clusters: # make data frame for each cluster
    c = df.loc[cluster]
    c = c.groupby(by = ['r']).mean().reset_index() # take average of data for repeated radii
    c["mu_corr"] = c["mu"] - c["AV"] # correct for extinction
    c["mu_min_corr"] = c["mu_min"] - c["AV"]
    c["mu_max_corr"] = c["mu_max"] - c["AV"]
    c["mu_phys"] = 10**(.4*(26.4 - c["mu_corr"])) # convert from mag/arcsec^2 to L_sun/pc^2
    c["mu_min_phys"] = 10**(.4*(26.4 - c["mu_min_corr"]))
    c["mu_max_phys"] = 10**(.4*(26.4 - c["mu_max_corr"]))
    c["e_mu_phys"] = abs(c["mu_phys"] - c["mu_min_phys"]) # compute error bars
    c["E_mu_phys"] = abs(c["mu_phys"] - c["mu_max_phys"])
    r_data, mu_data, mu_err, mu_Err = c["r"].to_numpy(), c["mu_phys"].to_numpy(), c["e_mu_phys"].to_numpy(), c["E_mu_phys"].to_numpy()
    output = np.column_stack((r_data, mu_data, mu_err, mu_Err))
    # save surface brightness profiles (with errors) in .txt file for each cluster
    np.savetxt(cluster, output) 

Define function for least squares fitting of a single cluster dataset:

In [ ]:
def fit_cluster(prof, file):
    '''
    Perform least-squares fitting for a given profile type and cluster.
    
    Parameters:
    -----------
    prof (str): Profile type
    file (str): Name of text file containing corrected cluster data
    
    Returns:
    --------
    Prints and returns best fit W0, r0, rho0, chi^2, fit parameter errors, half-light radius, and tidal radius
    
    '''
    data = np.genfromtxt(file, unpack = True) # import data with errors
    r_data, mu_data, mu_err, mu_Err = data[0], data[1], data[2], data[3] 
    stat_list = []
    params_list = []
    param_error_list = []
    
    W0_list = np.arange(1, 10.1, .1) # iterate over all template profiles
    for i in W0_list:
        if prof == "K":
            prof_string = "king"
        if prof == "W":
            prof_string = "wilson"
        # load template
        r, mu = np.genfromtxt("{}_{}".format(prof_string, str(int(i*10))), unpack = True)[:2]
        
        def model(r_data, a, b): # define model, which is linear rescaling of current template
            r_loc = r*a
            mu_loc = mu*b
            poly = make_interp_profile(r_loc, mu_loc)
            return poly(r_data)
        
        # perform fit and obtain parameters w/errors
        popt, pcov = curve_fit(model, r_data, mu_data, [1, 1], sigma=np.mean([mu_err, mu_Err], axis = 0), bounds = (0, np.inf), method = "trf")
        errs = np.sqrt(np.diag(pcov))
        params_list.append(popt)
        param_error_list.append(errs)
        obs = model(r_data, popt[0], popt[1]) 
        resid = obs - mu_data # residuals 
        chi_sq = np.sum((resid**2)/mu_data) # chi-squared
        stat_list.append(chi_sq)
        
    params = np.array(params_list)
    param_error = np.array(param_error_list)
    stats = np.array(stat_list)
    mask = (stats == min(stats)) # pick template/fit that yielded lowest chi-squared value
    best_W0, best_params, best_stat, best_error = W0_list[mask], params[mask][0], stats[mask], param_error[mask][0]

    r, mu = np.genfromtxt("{}_{}".format(prof_string, str(int(best_W0*10))), unpack = True)[:2]
    r_fit, mu_fit = r*best_params[0], mu*best_params[1] # apply best fit parameters to relevant template
    inv_poly2D = make_interp_profile(mu_fit, r_fit)
    Rh = inv_poly2D(.5*mu_fit[0]) # half-light radius
    Rt = r_fit[-1] # tidal radius
    print("%s fit to %s profile with: W0 = %.1f, a = %.3E, b = %.3E, chi_sq = %.3E. Rh = %.2f, Rt = %.2f"%(file, prof_string, best_W0, best_params[0], best_params[1], best_stat, Rh, Rt))
    return best_W0, best_params, best_stat, best_error, Rh, Rt

The cell below performs and plots profile fits for all cluster datasets. The lists initialized at the beginning will be populated with fit parameters and other relevant quantities for later tabulation.

We note again that we are performing these tasks simultaneously to mitigate unnecessary memory usage.

In [ ]:
# initialize parameter lists
W0_k_list = []
R_k_list = []
rho_k_list = []
chi_k_list = []
errR_k_list = []
errrho_k_list = []
Rh_k_list = []
Rt_k_list = []

W0_w_list = []
R_w_list = []
rho_w_list = []
chi_w_list = []
errR_w_list = []
errrho_w_list = []
Rh_w_list = []
Rt_w_list = []

Rlast_list = []

for i in clusters: # iterate over all clusters
    
    # load data 
    data = np.genfromtxt(i, unpack = True)
    r_data, mu_data, mu_err, mu_Err = data[0], data[1], data[2], data[3]
    
    # perform fit
    W0_k, params_k, chi_k, error_k, Rh_k, Rt_k = fit_cluster("K", i)
    W0_w, params_w, chi_w, error_w, Rh_w, Rt_w = fit_cluster("W", i)
    
    # append parameters to lists
    W0_k_list.append(W0_k)
    R_k_list.append(params_k[0])
    rho_k_list.append(params_k[1])
    chi_k_list.append(chi_k)
    errR_k_list.append(error_k[0])
    errrho_k_list.append(error_k[1])
    Rh_k_list.append(Rh_k)
    Rt_k_list.append(Rt_k)
    
    W0_w_list.append(W0_w)
    R_w_list.append(params_w[0])
    rho_w_list.append(params_w[1])
    chi_w_list.append(chi_w)
    errR_w_list.append(error_w[0])
    errrho_w_list.append(error_w[1])
    Rh_w_list.append(Rh_w)
    Rt_w_list.append(Rt_w)
    
    Rlast_list.append(r_data[-1]) # append twice since value is same for both profile types
    Rlast_list.append(r_data[-1])
    
        
    # plotting
    fig, ax = plt.subplots(figsize=(5,5))
    
    # print fitting parameters in plots
    textstr1 = '$x_{K} = (%.1f, %.2f, %.2f)$  \n $\chi^{2}_{K} = %.2f$'%(W0_k, params_k[0], params_k[1], chi_k)
    textstr2 = '$x_{W} = (%.1f, %.2f, %.2f)$  \n $\chi^{2}_{W} = %.2f$'%(W0_w, params_w[0], params_w[1], chi_w)
    props = dict(alpha=0)
        
    ax.text(0.05, 0.40, textstr1, transform=ax.transAxes, fontsize=16, verticalalignment='top', bbox=props)
    ax.text(0.05, 0.2, textstr2, transform=ax.transAxes, fontsize=16, verticalalignment='top', bbox=props)
    ax.text(0.08, 0.55, i, transform=ax.transAxes, fontsize=20, verticalalignment='top', bbox=props)
    
    # call best templates
    r_k, mu_k = np.genfromtxt("{}_{}".format('king', str(int(W0_k*10))), unpack = True)[:2]
    r_w, mu_w = np.genfromtxt("{}_{}".format('wilson', str(int(W0_w*10))), unpack = True)[:2]
    
    ax.set_yscale("log")
    ax.set_xscale("log")
    
    # plot cluster data with errors
    ax.scatter(r_data, mu_data, zorder = 5)
    ax.errorbar(r_data, mu_data, yerr = np.vstack((mu_err, mu_Err)), fmt="none", capsize = 2, zorder = 1, alpha = .5)
    
    ax.set_ylabel("$\mu_{V}$ [$L_{\odot, V}$/pc$^{2}$]", fontsize = 14)
    ax.set_xlabel("R [arcsec]", fontsize = 14)
    
    ax.tick_params(axis='x', labelsize= 14)
    ax.tick_params(axis='y', labelsize= 14)
      
    # apply fit parameters to templates
    r_k *= params_k[0] 
    mu_k *= params_k[1]
    
    r_w *= params_w[0] 
    mu_w *= params_w[1]
    
    # limits, colors, save plots
    ax.plot(r_k, mu_k, color = "black")
    ax.plot(r_w, mu_w, color = "black", ls = "--")
    ax.set_ylim(mu_k[-1], mu_k[0]*10)
    ax.set_xlim(r_k[0], r_k[-1]*10)
    plt.show()
    plt.tight_layout()
    #fig.savefig(i+"_fits", dpi = 400, bbox_inches='tight')

Make data frame for fit parameters and other relevant quantities for each cluster.

In [ ]:
df_list = []
dist_list = []

def arc_to_pc(arc, dist): # convert arcseconds to parsecs, input dist in kpc
    size = (dist*1e3)*(arc/206264.806)
    return size

for i in range(len(clusters)): # iterate over each cluster
    k_list = [clusters[i], "King", chi_k_list[i][0], W0_k_list[i][0], R_k_list[i], errR_k_list[i], rho_k_list[i], errrho_k_list[i], float(Rh_k_list[i]), Rt_k_list[i]]
    w_list = [clusters[i], "Wilson", chi_w_list[i][0], W0_w_list[i][0], R_w_list[i], errR_w_list[i], rho_w_list[i], errrho_w_list[i], float(Rh_w_list[i]), Rt_w_list[i]]
    
    if "LMC" in clusters[i]: # LMC distance is 50.1 kpc
        dist_list.append(50.1)
        dist_list.append(50.1)
        
    if "SMC" in clusters[i]: # SMC distance is 60 kpc
        dist_list.append(60)
        dist_list.append(60)
        
    if "FORNAX" in clusters[i]: # FORNAX distance is 137 kpc
        dist_list.append(137)
        dist_list.append(137)
    
    # make data frame for each cluster
    df = pd.DataFrame([k_list, w_list], columns=['Cluster', 'Profile', 'chi_sq', 'W0', 'R0', 'err_R0', 'rho0', 'err_rho0','Rh', 'Rt'])
    df = df.set_index(df.columns[0])
    df_list.append(df)

# concatenate all data frames
merged = pd.concat(df_list)
merged['Rlast'] = Rlast_list # radial extent of data
merged['dist'] = dist_list
merged['Rh_kpc'] = arc_to_pc(merged["Rh"], merged["dist"]) # convert to physical sizes
merged['Rt_kpc'] = arc_to_pc(merged["Rt"], merged["dist"])
merged['Rlast_kpc'] = arc_to_pc(merged["Rlast"], merged["dist"])
merged = merged.round(decimals = 2)
merged.to_latex() # output as latex table

This final cell is for the assessment of Goodness-of-fit of each profile type compared to the radial extent of the cluster data. The analysis will be conducted using the same $\Delta = (\chi_{W}^{2}  - \chi_{K}^{2})/(\chi_{W}^{2}  + \chi_{K}^{2})$ metric as considered in Miocchi et al. 2013. For each cluster, this metric will be plotted against the radial extent of the corresponding dataset scaled by the half-light radius obtained from the fit. The King half-light radius will be used in all cases since we see this quantity is relatively similar for both profile types.

In [ ]:
df_k = merged[merged['Profile'] == "King"] # obtain King chi-squared
Rh_k = df_k["Rh_kpc"] # half-light radii
Rlast = df_k["Rlast_kpc"].to_numpy() # radial extent of data
chi_k = df_k["chi_sq"].to_numpy()

df_w = merged[merged['Profile'] == "Wilson"] # obtain Wilson chi-squared
chi_w = df_w["chi_sq"].to_numpy()

delta = (chi_w - chi_k)/(chi_w + chi_k) # calculate delta
r_scaled = Rlast/Rh_k # scale radial extents

# plot
fig, ax = plt.subplots()
ax.tick_params(axis='both', labelsize=14)
plt.ylim(-.4, .2)
plt.scatter(r_scaled, delta)
plt.axhline(y=0, color='black', linestyle='--')
plt.title("Profile Goodness-of-Fit vs. Radial Extent of Data")
plt.ylabel("$\Delta = (\chi_{W}^{2}-\chi_{K}^{2})/(\chi_{W}^{2}+\chi_{K}^{2})$", fontsize = 14)
plt.xlabel("$r_{last}/r_{h}$", fontsize = 14)
plt.tight_layout()
fig.savefig('GOFplot', dpi = 200, bbox_inches = 'tight')